In [1]:
import sys
import pickle
import tracemalloc
import timeit
from time import time_ns
import psutil
import os

In [8]:
pip install pyavl3

In [9]:
import pyavl3

###Dados

In [ ]:
arquivos1 = ['/content/dados/alue2087.stp','/content/dados/alue2105.stp','/content/dados/alue3146.stp','/content/dados/alue5067.stp','/content/dados/alue5345.stp','/content/dados/alue5623.stp','/content/dados/alue5901.stp','/content/dados/alue6179.stp','/content/dados/alue6457.stp','/content/dados/alue6735.stp','/content/dados/alue6951.stp','/content/dados/alue7065.stp','/content/dados/alue7066.stp','/content/dados/alue7080.stp', '/content/dados/alue7229.stp']

In [ ]:
arquivos2 = ['/content/dados/alut0787.stp','/content/dados/alut0805.stp','/content/dados/alut1181.stp','/content/dados/alut2010.stp','/content/dados/alut2288.stp','/content/dados/alut2566.stp','/content/dados/alut2610.stp','/content/dados/alut2625.stp','/content/dados/alut2764.stp']

In [2]:
arquivos3 = ['/content/dados/dmxa0296.stp','/content/dados/dmxa0368.stp','/content/dados/dmxa0454.stp','/content/dados/dmxa0628.stp','/content/dados/dmxa0734.stp','/content/dados/dmxa0848.stp','/content/dados/dmxa0903.stp','/content/dados/dmxa1010.stp','/content/dados/dmxa1109.stp','/content/dados/dmxa1200.stp','/content/dados/dmxa1304.stp','/content/dados/dmxa1516.stp','/content/dados/dmxa1721.stp','/content/dados/dmxa1801.stp']

##Função do algoritmo de Prim

In [3]:
def prim(graph, start_node):  # Define a função prim
    mst = []  # Inicializa a árvore geradora mínima como uma lista vazia
    visited = set([start_node])  # Inicializa o conjunto de nós visitados com o nó inicial
    avl_tree = pyavl3.AVLTree()  # Cria uma árvore AVL vazia

    mem_before = process_memory() #Pega a memória inicial
    # Cria uma lista de arestas do nó inicial e seus custos
    edges = [
        (cost, start_node, to)
        for to, cost in graph[start_node].items()
    ]

    # Insere todas as arestas na árvore AVL com o custo como chave
    for edge in edges:
        avl_tree[edge] = edge[0]

    # Continua enquanto houver arestas na árvore AVL
    while len(avl_tree) > 0:
        # Obtém e remove a aresta de menor custo da árvore AVL
        cost, frm, to = avl_tree._get_min(avl_tree.root).key
        avl_tree.pop((cost, frm, to))

        # Se o nó 'to' ainda não foi visitado
        if to not in visited:
            visited.add(to)  # Adiciona o nó 'to' ao conjunto de nós visitados
            mst.append((frm, to, cost))  # Adiciona a aresta à árvore geradora mínima

            # Para cada nó adjacente ao nó 'to'
            for to_next, cost2 in graph[to].items():
                # Se o nó adjacente ainda não foi visitado
                if to_next not in visited:
                    # Adiciona a aresta à árvore AVL
                    avl_tree[(cost2, to, to_next)] = cost2
    mem_after = process_memory() #Pega a memória final
    mem_used = mem_after - mem_before  #Calcula a memória consumida em bytes

    return mst, mem_used  # Retorna a árvore geradora mínima

##Função para ler o arquivo .stp e traduzir para uma estrutura de dicionário representando um grafo

In [4]:
def read_stp(filename):
    graph = {}
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith('E '):
                data = line.strip().split()
                node1 = int(data[1])
                node2 = int(data[2])
                cost = float(data[3])
                if node1 not in graph:
                    graph[node1] = {}
                if node2 not in graph:
                    graph[node2] = {}
                graph[node1][node2] = cost
                graph[node2][node1] = cost
    return graph

##Função principal que usa as anteriores para ler o arquivo de entrada, modelar o grafo e retornar a MST

In [5]:
def main():
    filename = '/content/dmxa0296.stp'
    graph = read_stp(filename)
    mst, _ = prim(graph, 1)
    print(mst) #A Mst é retornada como uma lista de tuplas, cada tupla contém a aresta que é representada pelos seus vértices interligados e respectivo peso

In [ ]:
if __name__ == "__main__":
    main()

[(1, 2, 5.0), (1, 14, 13.0), (14, 15, 5.0), (15, 27, 13.0), (27, 26, 5.0), (27, 28, 5.0), (28, 29, 5.0), (29, 30, 5.0), (30, 31, 5.0), (31, 32, 5.0), (32, 33, 5.0), (33, 34, 5.0), (27, 40, 13.0), (40, 39, 5.0), (40, 41, 5.0), (41, 42, 5.0), (42, 43, 5.0), (43, 44, 5.0), (44, 45, 5.0), (45, 46, 5.0), (46, 47, 5.0), (47, 48, 5.0), (48, 49, 5.0), (49, 50, 5.0), (50, 51, 5.0), (51, 52, 5.0), (29, 16, 13.0), (16, 17, 5.0), (17, 18, 5.0), (18, 19, 5.0), (19, 20, 5.0), (20, 21, 5.0), (16, 3, 13.0), (3, 4, 5.0), (4, 5, 5.0), (5, 6, 5.0), (6, 7, 5.0), (7, 8, 5.0), (8, 9, 5.0), (9, 10, 5.0), (10, 11, 5.0), (11, 12, 5.0), (12, 13, 5.0), (10, 22, 13.0), (22, 23, 5.0), (23, 24, 5.0), (24, 25, 5.0), (22, 35, 13.0), (35, 36, 5.0), (36, 37, 5.0), (37, 38, 5.0), (39, 53, 13.0), (53, 54, 5.0), (54, 55, 5.0), (55, 56, 5.0), (56, 57, 5.0), (57, 58, 5.0), (58, 59, 5.0), (47, 60, 13.0), (60, 61, 5.0), (61, 62, 5.0), (62, 63, 5.0), (63, 64, 5.0), (64, 65, 5.0), (53, 66, 13.0), (66, 67, 5.0), (67, 68, 5.0), (

##Função para medir a memória gasta na execução do algoritmo


In [6]:
def process_memory():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss

##Funções para automatizar o treino e plotagem de gráficos

In [10]:
def avl_dataAuto(arquivos):
  avl_data = {"memoria":{}, "tempo":{}}

  for arquivo in (arquivos):
    graph = read_stp(arquivo)

    start_time = time_ns()
    mst, totalMemory = prim(graph, 1)
    end_time = time_ns()

    elapsed_time = end_time - start_time

    avl_data['memoria'][arquivo] = totalMemory
    avl_data['tempo'][arquivo] = elapsed_time


  return avl_data

dic_avl = avl_dataAuto(arquivos3)

with open("avl.pkl", "wb") as f:
    # Usa a função dump para salvar o dicionário no arquivo
    pickle.dump(dic_avl, f)

In [ ]:
dic_avl

In [ ]:
nome_arquivo = 'avl.pkl'
with open(nome_arquivo, 'rb') as arquivo:
    avl_time = pickle.load(arquivo)

In [ ]:
import matplotlib.pyplot as plt

x = range(1, len(avl_time) + 1)
y = avl_time.values()

plt.plot(x, y)
plt.title('Tempo gasto na execução de Prim usando AVL')
plt.xlabel('Dados')
plt.ylabel('Tempo(ns)')
plt.xticks(x)
plt.show()

In [ ]:
x = range(1, len(avl_space) + 1)
y = avl_space.values()

plt.plot(x, y)
plt.title('Memória gasta na execução de Prim usando AVL')
plt.xlabel('Dados')
plt.ylabel('Memória(B)')
plt.xticks(x)
plt.show()


##Exemplo de como é lido e interpretado o grafo de um arquivo de entrada

In [ ]:
filename = '/content/dmxa0296.stp'
graph = read_stp(filename)

In [ ]:
for to, cost in graph[1].items():
  print(f"to: {to}; cost: {cost}")

to: 2; cost: 5.0
to: 14; cost: 13.0


In [ ]:
#avl_tree.update(graph)
avl_tree = pyavl3.AVLTree()

# Adiciona alguns nós à árvore AVL
avl_tree[1] = 2
avl_tree[2] = 4
avl_tree[3] = 5
avl_tree[4] = 3

In [ ]:
print(avl_tree)

<AVL {3: 5, 2: 4, 4: 3}>


In [ ]:
avl_tree.pop(1) #Remove a chave e retorna o valor

2

In [ ]:
avl_tree._get_min(avl_tree.root).value #valor na menor chave, mas trocar value por key retorna a mínima chave

2

In [ ]:
#Encontrar o mínimo valor na arvore
def find_min(tree):
    node = tree.root
    while node.left is not None:
        node = node.left
    return node.key, node.value

<Node key=54, value={40: 13.0, 53: 5.0, 55: 5.0, 67: 13.0}, height=6>

In [ ]:
avl_tree.copy

{1: 5.0, 15: 13.0}